# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/installation.md) before you start.

# Imports

## Env setup

In [92]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
#import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from urllib.request import urlretrieve
from scipy.misc import imsave

from utils import label_map_util
from utils import visualization_utils as vis_util
from global_config import USE_GRAYSCALE
from IPython.display import display, clear_output

# This is needed to display the images.
%matplotlib inline

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

In [13]:
# Pretrained model files are located in /prog/models/exported
MODEL_NAME = '/prog/models/exported/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/prog/data', 'object-detection.pbtxt')

# Just arabic
NUM_CLASSES = 1

## Download Model

In [14]:
#opener = urllib.request.URLopener()
#opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
#tar_file = tarfile.open(MODEL_FILE)
#for file in tar_file.getmembers():
#  file_name = os.path.basename(file.name)
#  if 'frozen_inference_graph.pb' in file_name:
 #   tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [15]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [16]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [17]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  print(im_width, im_height)
  all_ints = np.array(image, dtype=np.uint8)
  print(all_ints.shape)
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [274]:
#url = "http://cdn9.staztic.com/app/a/925/925149/arabic-english-dictionary-1-0-s-307x512.jpg"
#url = 'http://cdn9.staztic.com/app/a/4622/4622090/aljazeera-arabic-news-4-3-s-307x512.jpg'
#url = 'https://s.cafebazaar.ir/1/upload/screenshot/com.microweb.feeding1.jpg'
#url = "http://a5.mzstatic.com/us/r30/Purple71/v4/59/7a/5f/597a5f98-abe4-b518-f61c-3080114eea36/screen696x696.jpeg"
#url = "http://a5.mzstatic.com/us/r30/Purple4/v4/0c/98/ef/0c98ef57-a00c-62f5-1619-2a49fbde70ef/screen800x500.jpeg"
#url = 'http://cdn8.staztic.com/app/a/759/759076/arabic-dictionary-full-436339-2-s-307x512.jpg'
#url = 'https://cdn7.staztic.com/app/a/2836/2836767/arabic-turkish-translator-51-0-s-307x512.jpg'
url = 'http://img1.gtimg.com/sports/pics/hv1/231/173/2205/143424471.jpg'

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/prog/test_images'
test_file_location = os.path.join(PATH_TO_TEST_IMAGES_DIR,"test.jpg")
urlretrieve(url, test_file_location)
TEST_IMAGE_PATHS = [test_file_location]
#TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
#TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR,"arabic-fortune-teller-6-0-s-307x512.jpg")]
print(TEST_IMAGE_PATHS)
# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)
#IMAGE_SIZE = (24, 16)


['/prog/test_images/test.jpg']


In [ ]:
display(Image.open(TEST_IMAGE_PATHS[0]))


In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    for image_path in TEST_IMAGE_PATHS:
      
      image = Image.open(image_path)
      if USE_GRAYSCALE:
        image = image.convert('L').convert('RGB')
            
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      #print(boxes[0][0:2]) 
      #boxes=[boxes[0][0:2]]
      #boxes = 
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=.50)
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)
        